In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

subway_line = pd.read_csv('/gdrive/MyDrive/DScover 2학기 프로젝트/서울교통공사 지하철역 위치정보.csv')
airbnb = pd.read_excel('/gdrive/MyDrive/DScover 2학기 프로젝트/숙소 전체_중복값 제거.xlsx')

In [ ]:
# 에어비앤비 위치정보 처리

airbnb['location'] = airbnb['location'].apply(lambda x: eval(x))

lat_lon = []

for x, y in airbnb['location'].to_list():
  x = float(x)
  y = float(y)
  lat_lon.append((x, y))

airbnb['locations'] = lat_lon

In [ ]:
# 서울 지하철역 위치정보 처리
subway_line = subway_line.sort_values(by=['위도', '경도'], ignore_index=True) # 위도/경도 기준 정렬
subway_line['location'] = [(subway_line.iloc[i, -2], subway_line.iloc[i, -1]) for i in range(len(subway_line))]

In [ ]:
from haversine import haversine

# 거리 계산 함수
def getDist(p1, p2):
  return haversine(p1, p2, unit='km')

# 지하철역별 거리 반환
def dac(airbnb, subway):
  result = []
  for i in range(len(airbnb)):
    distances = []
    for j in range(len(subway)):
      distances.append(getDist(airbnb[i], subway[j]))
    result.append(distances)
  return result

airbnb['distances'] = dac(airbnb['locations'].to_list(), subway_line['location'].to_list())

In [ ]:
# 최소거리 계산 및 인근 지하철역 정보 저장
def min_index(df):
  min_distance = []
  subway = []
  for d in df['distances']:
    min_distance.append(min(d)) # 최소 거리
    min_i = d.index(min(d))
    subway.append(subway_line.iloc[min_i, 1]) # 최소 거리 지하철역
  df['min_distance'] = min_distance
  df['subway'] = subway

min_index(airbnb)

In [ ]:
# 최종 데이터셋 저장
airbnb.drop(['location'], axis=1, inplace=True)
airbnb.drop(['distances'], axis=1, inplace=True)
airbnb.to_csv('인근 지하철역 정보 추가.csv', index=False, encoding='utf-8-sig')